In [ ]:
# default_exp directed_diffusion_maps 
from nbdev.showdoc import *
import numpy as np
from sklearn.metrics import pairwise_distances
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Perrault-Jonas2011's Directed Graph Embeddings
### Re-implemented for benchmarking against our techniques

[Perrault-Jonas 2011 - Directed Graph Embeddings](x-devonthink-item://58E9930C-8189-403E-8EF0-8D0D545EDA54) gives one compelling way to embed directed graphs. Whereas work on the Magnetic Laplacian and other means of working with directed graphs approaches the problem from a graph-theoretic viewpoint, Perrault and Jonas view the problem as a *manifold problem*: just as diffusion maps embed undirected graphs as if they are points drawn from a manifold, the proposed algorithm embeds points as if they come from a manifold covered by a flowing vector field.

Think of it as diffusion maps for manifolds with flow.

This approach has the merits of theoretical cleanliness, but inherits some practical issues from diffusion maps: 

1. The embedding of the points has *nothing to do* with the flow. Diffusion maps reproduce the manifold without any consideration of the directed information.

2. Diffusion coordinates are not optimized for visualization. Diffusion vectors represent branches of the manifold, but each branch is given its own dimension.

In [ ]:
#export
import logging
def directed_diffusion_map(W, t = 1, kernel_type = "anisotropic", alpha = 0.5, sigma = "automatic", k = 10):
  
  Dists = distance_matrix(X)
  if sigma == "automatic":
    # Heuristic for sigma: median of the distance to the kth nearest neighbor
    sigma = np.median(np.partition(Dists,k)[:,k])
    print("using sigma = ",sigma)
  W = anisotropic_kernel(Dists, sigma=sigma, alpha = alpha)
  P_symmetric, D = diffusion_matrix(W, symmetric=True, return_degree=True)
  diff_map = diffusion_coordinates(P_symmetric, D, t = t)
  diff_map = diff_map[:,::-1]
  diff_map = diff_map.T
  return diff_map